In [18]:
def generate_char_list(start_char, end_char, step):
    """
    Generates a list of characters from start_char to end_char with a given step.
    
    Parameters:
    start_char (str): The starting character.
    end_char (str): The ending character.
    step (int): The step between characters.
    
    Returns:
    list: List of characters.
    """
    start = ord(start_char.upper())
    end = ord(end_char.upper())
    return [chr(i) for i in range(start, end + 1, step)]

# Contoh penggunaan
start_char = 'D'
end_char = 'M'
step = 2

# result = generate_char_list(start_char, end_char, step)
# print(result)  # Output: ['D', 'F', 'H']

col_absen = generate_char_list('D','M',2)
for char in generate_char_list('O','X',2):
    col_absen.append(char)

print(col_absen)

['D', 'F', 'H', 'J', 'L', 'O', 'Q', 'S', 'U', 'W']


In [19]:
import pandas as pd
from openpyxl import load_workbook

def edit_excel(file_path, dataframe, kolom_ujian, target_path):
    # Load workbook and select the active worksheet
    workbook = load_workbook(file_path)
    sheet = workbook.active
    
    # Define the starting row and ending row
    start_row = 18
    end_row = 45
    
    # Iterate over the rows and update the values from the dataframe
    for idx, row in enumerate(range(start_row, end_row + 1)):
        if idx < len(dataframe):
            sheet[f'B{row}'] = dataframe.iloc[idx]['NPM']
            sheet[f'C{row}'] = dataframe.iloc[idx]['Nama Lengkap']
            sheet[f'D{row}'] = dataframe.iloc[idx][kolom_ujian]
    
    # Save the workbook
    workbook.save(target_path)

def edit_excel_2(file_path, dataframe, target_path, uts_col, uas_col, praktikum_col):
    # Load workbook and select the active worksheet
    workbook = load_workbook(file_path)
    sheet = workbook.active
    
    # Define the starting row and ending row
    start_row = 13
    end_row = 40
    
    # Iterate over the rows and update the values from the dataframe
    for idx, row in enumerate(range(start_row, end_row + 1)):
        if idx < len(dataframe):
            col_absen = generate_char_list('D','M',2)
            for char in generate_char_list('O','X',2):
                col_absen.append(char)

            col_tugas = generate_char_list('E','M',2)
            for char in generate_char_list('P','X',2):
                col_tugas.append(char)

            for char in col_absen:
                sheet[f'{char}{row}'] = dataframe.iloc[idx]["NILAI ABSEN"]

            for ii, char in enumerate(col_tugas):
                sheet[f'{char}{row}'] = dataframe.iloc[idx][praktikum_col[ii]]

            sheet[f'B{row}'] = dataframe.iloc[idx]['NPM']
            sheet[f'C{row}'] = dataframe.iloc[idx]['Nama Lengkap']
            sheet[f'N{row}'] = dataframe.iloc[idx][uts_col]
            sheet[f'Y{row}'] = dataframe.iloc[idx][uas_col]
            sheet[f'AA{row}'] = dataframe.iloc[idx][uts_col]
            sheet[f'AB{row}'] = dataframe.iloc[idx][uas_col]
            sheet[f'AE{row}'] = dataframe.iloc[idx][uts_col]
            sheet[f'AF{row}'] = dataframe.iloc[idx][uas_col]
            sheet[f'AC{row}'] = dataframe.iloc[idx]["NILAI ABSEN"] * 10
            sheet[f'AG{row}'] = dataframe.iloc[idx]["NILAI ABSEN"]
            sheet[f'AH{row}'] = dataframe.iloc[idx]["TOTAL NILAI(%)"]
            sheet[f'AD{row}'] = dataframe.iloc[idx]["TOTAL NILAI PRAKTIKUM(%)"]
            sheet[f'Z{row}'] = dataframe.iloc[idx]["TOTAL NILAI PRAKTIKUM"]
    
    # Save the workbook
    workbook.save(target_path)

"""
TUGAS PRAKTIKUM = 30%
UTS PRAKTIKUM = 30%
UAS PRAKTIKUM = 40%
"""

def rekapitulasi_nilai(path_file_nilai, path_file_uas, target_path_file):

    uts_col = "UTS Praktikum"
    uas_col = "UAS Praktikum"

    def rekap_nilai(tugas, uts, uas):
        absen = 100.0
        return (float(tugas) * 0.2) + (float(uts) * 0.3) + (float(uas) * 0.4) + (float(absen) * 0.1)

    def rekap_nilai_praktikum(list_nilai):
        return sum(list_nilai) / len(list_nilai)

    df = pd.read_csv(path_file_nilai, delimiter=",", on_bad_lines='skip')
    df_uas = pd.read_excel(path_file_uas)
    df = df.iloc[2:]
    df = df.reset_index(drop=True)

    nama_col = ['Nama Belakang', 'Nama Depan']
    for col in nama_col:
        df[col] = df[col].fillna("")

    df["Nama Lengkap"] = ""

    for i in range(len(df)):
        try:
            df["Nama Lengkap"][i] = str(str(df["Nama Depan"][i]) + " " + str(df['Nama Belakang'][i])).split("_")[1].upper()
        except:
            df["Nama Lengkap"][i] = str(str(df["Nama Depan"][i]) + " " + str(df['Nama Belakang'][i])).upper()
    
    all_col = ['Nama Belakang', 'Nama Depan', "Nama Lengkap", 'Alamat Email']
    for col in df.columns:
        if "Praktikum" in col or "Practicum" in col:
            all_col.append(col)

    df = df[all_col]

    df["NPM"] = [(df["Alamat Email"][i]).split("@")[0] for i in range(len(df))]
    df_uas["NPM"] = [str(df_uas["NPM"][i]) for i in range(len(df_uas))]
    df_merge = pd.merge(df, df_uas, on="NPM", how="inner")

    df[uas_col] = df_merge["Total"]
    df['NILAI ABSEN'] = 100.0
    df = df.fillna(0.0)

    praktikum_col = [col for col in df.columns if ("UTS" not in col) and ("UAS" not in col) and ("Praktikum" in col) or ("Practicum" in col)]
    df['TOTAL NILAI PRAKTIKUM'] = [sum([float(df[col][i]) for col in praktikum_col]) for i in range(len(df))]
    df['TOTAL NILAI PRAKTIKUM(%)'] = [rekap_nilai_praktikum([float(df[col][i]) for col in praktikum_col]) for i in range(len(df))]
    df['TOTAL NILAI(%)'] = [rekap_nilai(df['TOTAL NILAI PRAKTIKUM(%)'][i], df[uts_col][i], df[uas_col][i]) for i in range(len(df))]

    dir_target = path_file_nilai.split("/")[0]
    target_path = f"{dir_target}/Daftar Nilai UTS {dir_target}.xlsx"
    edit_excel("Daftar Nilai UTS.xlsx", df, uts_col, target_path)
    target_path = f"{dir_target}/Daftar Nilai UAS {dir_target}.xlsx"
    edit_excel("Daftar Nilai UAS.xlsx", df, uas_col, target_path)
    target_path = f"{dir_target}/Rekapitulasi Nilai Praktikum {dir_target}.xlsx"
    edit_excel_2("Rekapitulasi Nilai Praktikum.xlsx", df, target_path, uts_col, uas_col, praktikum_col)

    return df, df_merge
    # df.to_excel(target_path_file, index=False)

In [20]:
df, df2 = rekapitulasi_nilai("4B/Informatika_4B_Kecerdasan_Buatan.csv", "4B/Nilai UAS Praktikum Kecerdasan Buatan 4B.xlsx", "")

C:\Users\herutriana44\AppData\Local\Temp\ipykernel_12172\2582271439.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Nama Lengkap"][i] = str(str(df["Nama Depan"][i]) + " " + str(df['Nama Belakang'][i])).split("_")[1].upper()
C:\Users\herutriana44\AppData\Local\Temp\ipykernel_12172\2582271439.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Nama Lengkap"][i] = str(str(df["Nama Depan"][i]) + " " + str(df['Nama Belakang'][i])).split("_")[1].upper()
C:\Users\herutriana44\AppData\Local\Temp\ipykernel_12172\2582271439.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [21]:
df2

,Nama Belakang,Nama Depan,Nama Lengkap,Alamat Email,Praktikum 1,Praktikum 2,Praktikum 3,Praktikum 4,Assignment Practicum 5 - AI Study Case,UTS Praktikum,...,Praktikum 8,Praktikum 9,Praktikum 10,NPM,Kelompok,Nama,Presentasi(15),Tanya Jawab(5),Kode(80),Total
0,QOBUS ZAIDAN,2B_MAGROZAN,MAGROZAN QOBUS ZAIDAN,2210631170127@student.unsika.ac.id,100.0,100.0,100.0,95.0,100.0,100.0,...,100.0,100.0,100.0,2210631170127,ORBA,Magrozan Qobus Z.,75,0,95,87.25
1,DINI KENCANA,4B_ARJUNA,ARJUNA DINI KENCANA,2210631170009@student.unsika.ac.id,100.0,90.0,90.0,100.0,80.0,75.0,...,100.0,100.0,100.0,2210631170009,SUDARSEP,Arjuna Dini Kencana,75,0,90,83.25
2,SARIFATUN NISA,4B_AUFA,AUFA SARIFATUN NISA,2210631170010@student.unsika.ac.id,100.0,95.0,100.0,100.0,100.0,90.0,...,100.0,100.0,100.0,2210631170010,PASS,Aufa Sarifatun Nisa,75,0,90,83.25
3,ADYATMA PRATAMA,4B_AZHAR,AZHAR ADYATMA PRATAMA,2210631170011@student.unsika.ac.id,95.0,95.0,100.0,100.0,90.0,80.0,...,100.0,100.0,100.0,2210631170011,ORBA,Azhar Adyatma P,75,0,95,87.25
4,Azizi,4B_Bhanu,BHANU AZIZI,2210631170013@student.unsika.ac.id,90.0,95.0,100.0,90.0,90.0,80.0,...,100.0,100.0,100.0,2210631170013,ORBA,Bhanu Azizi,95,100,95,95.25
5,AMELIA PRAMESWARI,4B_CINDY,CINDY AMELIA PRAMESWARI,2210631170015@student.unsika.ac.id,100.0,95.0,100.0,50.0,90.0,70.0,...,100.0,100.0,100.0,2210631170015,PACINURA,Cindy Amelia,75,0,90,83.25
6,AL GHIFARY,4B_DAFFA,DAFFA AL GHIFARY,2210631170016@student.unsika.ac.id,NaN,95.0,100.0,100.0,80.0,70.0,...,100.0,100.0,100.0,2210631170016,SUDARSEP,Daffa Al Ghifary,75,0,90,83.25
7,SUTAN FATHIR,4B_MUHAMMAD,MUHAMMAD SUTAN FATHIR,2210631170090@student.unsika.ac.id,95.0,85.0,100.0,80.0,80.0,70.0,...,100.0,100.0,100.0,2210631170090,SUDARSEP,M. Sutan Fathir,95,100,90,91.25
8,,4B_NUR SALAMAH AZZAHRAH,NUR SALAMAH AZZAHRAH,2210631170095@student.unsika.ac.id,100.0,90.0,100.0,50.0,80.0,80.0,...,100.0,100.0,100.0,2210631170095,PACINURA,Nur Salamah,95,100,90,91.25
9,Pasha Marcellina,4B_Nabilla,NABILLA PASHA MARCELLINA,2210631170091@student.unsika.ac.id,100.0,90.0,100.0,80.0,80.0,80.0,...,90.0,100.0,100.0,2210631170091,PACINURA,Nabilla Pasha,75,0,90,83.25


In [22]:
df

,Nama Belakang,Nama Depan,Nama Lengkap,Alamat Email,Praktikum 1,Praktikum 2,Praktikum 3,Praktikum 4,Assignment Practicum 5 - AI Study Case,UTS Praktikum,...,Praktikum 7,Praktikum 8,Praktikum 9,Praktikum 10,NPM,UAS Praktikum,NILAI ABSEN,TOTAL NILAI PRAKTIKUM,TOTAL NILAI PRAKTIKUM(%),TOTAL NILAI(%)
0,QOBUS ZAIDAN,2B_MAGROZAN,MAGROZAN QOBUS ZAIDAN,2210631170127@student.unsika.ac.id,100.0,100.0,100.0,95.0,100.0,100.0,...,80.0,100.0,100.0,100.0,2210631170127,87.25,100.0,970.0,97.0,94.30
1,DINI KENCANA,4B_ARJUNA,ARJUNA DINI KENCANA,2210631170009@student.unsika.ac.id,100.0,90.0,90.0,100.0,80.0,75.0,...,100.0,100.0,100.0,100.0,2210631170009,83.25,100.0,945.0,94.5,84.70
2,SARIFATUN NISA,4B_AUFA,AUFA SARIFATUN NISA,2210631170010@student.unsika.ac.id,100.0,95.0,100.0,100.0,100.0,90.0,...,100.0,100.0,100.0,100.0,2210631170010,83.25,100.0,980.0,98.0,89.90
3,ADYATMA PRATAMA,4B_AZHAR,AZHAR ADYATMA PRATAMA,2210631170011@student.unsika.ac.id,95.0,95.0,100.0,100.0,90.0,80.0,...,100.0,100.0,100.0,100.0,2210631170011,87.25,100.0,980.0,98.0,88.50
4,Azizi,4B_Bhanu,BHANU AZIZI,2210631170013@student.unsika.ac.id,90.0,95.0,100.0,90.0,90.0,80.0,...,100.0,100.0,100.0,100.0,2210631170013,95.25,100.0,955.0,95.5,91.20
5,AMELIA PRAMESWARI,4B_CINDY,CINDY AMELIA PRAMESWARI,2210631170015@student.unsika.ac.id,100.0,95.0,100.0,50.0,90.0,70.0,...,100.0,100.0,100.0,100.0,2210631170015,83.25,100.0,920.0,92.0,82.70
6,AL GHIFARY,4B_DAFFA,DAFFA AL GHIFARY,2210631170016@student.unsika.ac.id,0.0,95.0,100.0,100.0,80.0,70.0,...,100.0,100.0,100.0,100.0,2210631170016,83.25,100.0,860.0,86.0,81.50
7,SUTAN FATHIR,4B_MUHAMMAD,MUHAMMAD SUTAN FATHIR,2210631170090@student.unsika.ac.id,95.0,85.0,100.0,80.0,80.0,70.0,...,100.0,100.0,100.0,100.0,2210631170090,91.25,100.0,940.0,94.0,86.30
8,,4B_NUR SALAMAH AZZAHRAH,NUR SALAMAH AZZAHRAH,2210631170095@student.unsika.ac.id,100.0,90.0,100.0,50.0,80.0,80.0,...,100.0,100.0,100.0,100.0,2210631170095,91.25,100.0,895.0,89.5,88.40
9,Pasha Marcellina,4B_Nabilla,NABILLA PASHA MARCELLINA,2210631170091@student.unsika.ac.id,100.0,90.0,100.0,80.0,80.0,80.0,...,100.0,90.0,100.0,100.0,2210631170091,83.25,100.0,925.0,92.5,85.80
